# Computer Vision in a nutshell

Workshop created for PyConPL 2019 conference.

### Setup

We will need:

- tensorflow
- numpy
- scipy
- opencv-python
- pillow
- h5py
- matplotlib
- keras
- imageai

See the *requirements.txt* file.

Also we will need the trained model for objects detection:
https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/resnet50_coco_best_v2.0.1.h5

### Working with images

In [ ]:
# Imports
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Load an image
image = cv2.imread("data//lena.png")

In [ ]:
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyWindow("Image")

In [ ]:
# Convert to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
cv2.imshow("Gray", gray_image)
cv2.waitKey(0)
cv2.destroyWindow("Gray")

### Smoothing image

In [ ]:
blurred = cv2.blur(image,(5,5))
blurred_gauss = cv2.GaussianBlur(image, (5,5), 0)
blurred_median = cv2.medianBlur(image, 5)

In [ ]:
cv2.imshow("Image", image)
cv2.imshow("Average", blurred)
cv2.imshow("Gaussian", blurred_gauss) 
cv2.imshow("Median", blurred_median)
cv2.waitKey(0)
cv2.destroyAllWindows()

### EXERCISE

Remove noise from bridge image (*bridge.jpg* in the data folder)

In [ ]:
# Type your solution here

### Edge detection

In [ ]:
# Edge detection
canny = cv2.Canny(blurred, 10, 30)
cv2.imshow("Canny with low thresholds", canny)

In [ ]:
canny2 = cv2.Canny(blurred, 50, 150)
cv2.imshow("Canny with high thresholds", canny2)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Histogram 

In [ ]:
# Histogram calculation
hist = cv2.calcHist(gray_image, [1], None,[256],[0,256])
plt.plot(hist)
plt.xlim([0,256])
plt.show()

In [ ]:
# Histogram equalization
equ = cv2.equalizeHist(gray_image)

In [ ]:
res = np.hstack((gray_image, equ)) # Stacking images side-by-side
cv2.imshow("Original and equalized", res)
cv2.waitKey(0)
cv2.destroyAllWindows()

### EXERCISE
Calculate histogram of the image afer equalization.

In [ ]:
# Type your solution here

### Thresholding

In [ ]:
ret, threshold = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
adaptive = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 1)
cv2.imshow('Original',gray_image)
cv2.imshow('Threshold',threshold)
cv2.imshow('Adaptive threshold',adaptive)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Face recognition

In [ ]:
# Create the haar cascade
casc_path = "data//haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(casc_path)

In [ ]:
# Read the image
image = cv2.imread("data//people.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
# Detect faces in the image
faces = face_cascade.detectMultiScale(
    gray,
    scaleFactor = 1.1,
    minNeighbors = 5,
    minSize = (30,30)
    )

In [ ]:
print("Found {0} faces!".format(len(faces)))

In [ ]:
# Draw a rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2.imshow("Faces found" ,image)
cv2.waitKey(0)
cv2.destroyWindow("Faces found")

### Webcam

In [ ]:
video_capture = cv2.VideoCapture(0)

In [ ]:
while True:
    # Capture frame-by-frame
    ret, image = video_capture.read()

    if not ret:
        break

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (30,30)
        )
    #print("The number of faces found = ", len(faces))

    for (x,y,w,h) in faces:
        cv2.rectangle(image, (x,y), (x+h, y+h), (0, 255, 0), 2)

    cv2.imshow("Faces found", image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# Release the capture
video_capture.release()
cv2.destroyAllWindows()

### Motion detection

In [ ]:
video_capture = cv2.VideoCapture(0)

# Read two frames, last and current, and convert current to gray.
ret, last_frame = video_capture.read()
ret, current_frame = video_capture.read()
gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

i = 0
while(True):
    last_frame = current_frame

    ret, current_frame = video_capture.read()
    gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    
    # Find the absolute difference between current and previous frame
    diff = cv2.absdiff(last_frame, current_frame)
    
    # Uncomment the below to see the difference values
    '''
    i += 1
    if i % 10 == 0:
    i = 0
    print np.mean(current_frame)
    print np.mean(diff)
    '''

    # If difference is greater than a threshold, that means motion detected.
    if np.mean(diff) > 10:
        print("Motion detected.")
        
    # Display the resulting frame
    cv2.imshow('Video',diff)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# Release the capture
video_capture.release()
cv2.destroyAllWindows()

### Recognition using deep neural networks

Simple example using ImageAI library

In [ ]:
from imageai.Detection import ObjectDetection
import os

execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath( os.path.join(execution_path , "resnet50_coco_best_v2.0.1.h5"))
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "test1.jpg"), output_image_path=os.path.join(execution_path , "imagenew.jpg"))

for eachObject in detections:
    print(eachObject["name"] , " : " , eachObject["percentage_probability"] )